# Import Libraries

In [1]:
import os
import json
from parser import parse_pdf
from indexing import generate_vector_index,get_retriever

/Users/swastikmaiti/Desktop/temp/reimagine_rag/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
<All keys matched successfully>


## Declare Paths

In [2]:
base_path = os.getcwd()  

In [3]:
pdf_base = os.path.join(base_path, "pdf")
parsed_results_base = os.path.join(base_path, "parsed_results")

# Parse PDF

In [4]:
pdf_name = "who_statistic.pdf"
pdf_path = os.path.join(pdf_base, pdf_name)
parsed_result = None

In [5]:
with open(pdf_path, "rb") as f:
    pdf_bytes = f.read()
file_name = os.path.basename(pdf_path)

parsed_result = await parse_pdf(pdf_bytes=pdf_bytes,file_name=file_name)

Started parsing the file under job_id 04128041-53f1-4f9e-995f-fd375b94ef0f


#### Save the Prased Result

In [6]:
parsed_output_file_name = "parsed_output.json"
parsed_output_path = os.path.join(parsed_results_base, parsed_output_file_name)
if parsed_result:
    os.makedirs(os.path.dirname(parsed_output_path), exist_ok=True)
    with open(parsed_output_path, "w", encoding="utf-8") as f:
        json.dump(parsed_result.model_dump(), f, ensure_ascii=False, indent=2)


# Create Vector Index

In [7]:
pdf_content_file_name = "pdf_content.json"
pdf_content_path = os.path.join(parsed_results_base, pdf_content_file_name)


chroma_db_path = os.path.join(base_path, "chroma_db")
collection_name = "collection1"


### Load Parsed Result

In [8]:
with open(parsed_output_path, "r", encoding="utf-8") as f:
    result = json.load(f)

In [9]:
generate_vector_index(pdf_result=result,pdf_content_path=pdf_content_path,chromadb_path=chroma_db_path,collection_name=collection_name)

Content saved to /Users/swastikmaiti/Desktop/temp/reimagine_rag/parsed_results/pdf_content.json


### Test Retriever

In [10]:
retriever = get_retriever(chromadb_path=chroma_db_path,collection_name=collection_name)
query = "What is the life expectancy of males in 2020?"
results = retriever.retrieve(query)

In [11]:
results

[NodeWithScore(node=TextNode(id_='493bc434-4d88-43a4-a55b-bf40f81aa96e', embedding=None, metadata={'page_no': 14}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='# 1.1 Life expectancy and healthy life expectancy\n## 1.1.1 Global trend\nSteady gain in global life expectancy at birth was observed from the turn of the millennium until the onset of the COVID-19 pandemic, up from 66.8 years (95% uncertainty interval¹ [UI]: 66.2–67.3 years) in 2000 to 73.1 (UI: 72.6–73.7) years in 2019, with men gaining 6.2 years from 64.4 (UI: 63.9–64.9) years to 70.6 (UI 70.1–71.2) years and women gaining 6.5 years from 69.2 (UI: 68.7–69.8) years to 75.7 (UI: 75.3–76.2) years during this period. In parallel, global HALE at birth increased from 58.1 (UI: 57.4–58.9) years in 2000 to 63.5 (UI: 62.7–64.3) years in 2019, with that for men increasing from 57.0 (UI: 56.3–57.6) years to 62.3 (UI: 61.7–63.1) years 

# Test Agent

In [12]:
from process_query import QueryProcessor

In [13]:
processor = QueryProcessor(
    pdf_content_path=pdf_content_path,
    chromadb_path=chroma_db_path,
    collection_name=collection_name,
)

In [14]:
response = await processor.process_query(query="What is the life expectancy of males in 2020?")

In [15]:
print(response)

Based on the provided context, the life expectancy of males in 2020 was approximately 70.5 years. This information is found in Figure 1.1, which details global trends in life expectancy at birth by sex from 2000 to 2021.
